In [3]:
import pandas as pd 
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect


# Load environment variables from .env file
load_dotenv()

db_host = os.getenv('db_host')
db_user = os.getenv('db_user')
db_password = os.getenv('db_password')
db_name = os.getenv('db_name')

# Create the SQLAlchemy engine using environment variables
engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")


print("SQLAlchemy engine created successfully!")

SQLAlchemy engine created successfully!


In [4]:
# Create an inspector
inspector = inspect(engine)

# Get all table names
tables = inspector.get_table_names()

print("Tables in the database:")
for table in tables:
    print(table)

Tables in the database:
branches
customers
denormalized_inventory
inventory
order_items
orders
product_suppliers
products
suppliers


In [5]:
# Create inspector
inspector = inspect(engine)

# Get all table names
tables = inspector.get_table_names()

print("Columns in each table:\n")

# Loop through all tables except 'denormalized_inventory'
for table in tables:
    if table == 'denormalized_inventory':
        continue

    print(f"Table: {table}")
    columns = inspector.get_columns(table)
    for col in columns:
        print(f"  - {col['name']} ({col['type']})")
    print("\n")

Columns in each table:

Table: branches
  - branch_id (VARCHAR(10))
  - branch_name (VARCHAR(100))
  - branch_location_code (VARCHAR(10))
  - branch_city (VARCHAR(50))
  - branch_phone (VARCHAR(20))
  - branch_operating_hours (VARCHAR(100))
  - created_at (DATETIME)
  - updated_at (DATETIME)


Table: customers
  - customer_id (INTEGER)
  - customer_name (VARCHAR(100))
  - customer_phone_number (VARCHAR(25))
  - customer_email (VARCHAR(100))
  - customer_address (VARCHAR(255))
  - customer_loyalty_points (INTEGER)
  - customer_since (DATE)
  - customer_last_order_date (DATE)
  - customer_total_orders (INTEGER)
  - customer_notes (TEXT)
  - customer_acquisition_channel (VARCHAR(50))
  - customer_segment (VARCHAR(50))
  - customer_gender (ENUM)
  - number_of_visits (INTEGER)
  - created_at (DATETIME)
  - updated_at (DATETIME)


Table: inventory
  - inventory_id (INTEGER)
  - branch_id (VARCHAR(10))
  - product_id (VARCHAR(10))
  - quantity_on_hand (DECIMAL(10, 2))
  - min_stock_level (DEC

In [6]:
# Select all customer data, limited to 10 rows
query = """
SELECT * FROM customers
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head(4)

,customer_id,customer_name,customer_phone_number,customer_email,customer_address,customer_loyalty_points,customer_since,customer_last_order_date,customer_total_orders,customer_notes,customer_acquisition_channel,customer_segment,customer_gender,number_of_visits,created_at,updated_at
0,1,Stephanie Brewer,+254723130927,benjamin@yahoo.com,"2813 John Drives\nKennethland, MD 06937",367,2020-10-25,2024-11-24,25,Couple drive you behind look within technology...,In-store Promotion,Wholesale,Other,46,2025-04-20 23:25:04,2025-04-20 23:25:04
1,2,Cheryl Cook,+254758142567,valerie@outlook.com,"61799 Hernandez Drives Apt. 921\nEwingtown, MD...",812,2024-05-18,2024-12-10,55,Individual visit paper agency subject. Recogni...,Referral,VIP,Male,2,2025-04-20 23:25:04,2025-04-20 23:25:04
2,3,Raymond Brooks,+254732791070,raymond@outlook.com,"246 Todd Lights\nSouth Alanbury, NM 45687",792,2024-07-29,2025-01-04,72,Note respond age stop early. Knowledge citizen...,Email Marketing,VIP,Male,40,2025-04-20 23:25:04,2025-04-20 23:25:04
3,4,Robert Wilkerson MD,+254748840323,javier@gmail.com,"473 Garza Field Apt. 554\nWest Daniel, DC 29047",459,2023-02-23,2025-01-08,27,Response piece measure. Easy determine safe ab...,Website,Corporate,Female,50,2025-04-20 23:25:04,2025-04-20 23:25:04


In [7]:
# Get product names and prices, sorted by price (descending)
query = """
SELECT product_name, unit_price 
FROM products
ORDER BY unit_price DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head(4)

,product_name,unit_price
0,Affogato,49.98
1,Sandwich (Chicken),48.98
2,Fruit Tart,48.33
3,Vanilla Milkshake,48.22


In [8]:
# Get branch names, cities, and location codes
query = """
SELECT branch_name, branch_city, branch_location_code 
FROM branches;
"""

df = pd.read_sql(query, engine)
df.head(4)

,branch_name,branch_city,branch_location_code
0,Java House Monrovia Street,Nairobi,BR100202
1,Java House Airport View,Nairobi,BR100550
2,Java House Embassy House,Nairobi,BR110339
3,ABC Place,Nairobi,BR118460


In [9]:
# Count the total number of orders
query = """
SELECT COUNT(*) AS total_orders 
FROM orders;
"""

df = pd.read_sql(query, engine)
df.head(4)

,total_orders
0,200


In [10]:
# Sum product quantities by branch, ordered by total stock
query = """
SELECT branch_id, 
       SUM(quantity_on_hand) AS total_stock
FROM inventory
GROUP BY branch_id
ORDER BY total_stock DESC;
"""

df = pd.read_sql(query, engine)
df.head()

,branch_id,total_stock
0,BR764354,996.73
1,BR958241,994.07
2,BR394644,989.12
3,BR432646,981.84
4,BR949521,967.76


In [11]:
# Join products with their suppliers
query = """
SELECT p.product_name, s.supplier_name 
FROM products p
JOIN product_suppliers ps ON p.product_id = ps.product_id
JOIN suppliers s ON ps.supplier_id = s.supplier_id;
"""

df = pd.read_sql(query, engine)
df.head(4)

,product_name,supplier_name
0,Iced Chai Frappe,Local Delights
1,Hot Chocolate,Local Delights
2,Affogato,Local Delights
3,Protein Shake,Local Delights


In [12]:
# Calculate total revenue by multiplying quantity ordered by unit price
query = """
SELECT oi.product_id, 
       SUM(oi.quantity * oi.unit_price_at_order) AS total_revenue
FROM order_items oi
GROUP BY oi.product_id
ORDER BY total_revenue DESC;
"""

df = pd.read_sql(query, engine)
df.head()

,product_id,total_revenue
0,P930129,2232.50
1,P239331,2162.37
2,P578372,2142.00
3,P553945,2057.12
4,P474663,2032.00


In [13]:
query = """
SELECT DISTINCT branch_id
FROM orders;
"""

df = pd.read_sql(query, engine)
df.head(4)

,branch_id
0,BR100202
1,BR100550
2,BR110339
3,BR118460


In [14]:
query = """
SELECT DISTINCT c.customer_name, c.customer_email
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.branch_id = 'BR100202';
"""

df = pd.read_sql(query, engine)
df.head()

,customer_name,customer_email
0,Courtney Owens,john@yahoo.com


In [15]:
# Get products with stock below reorder level
query = """
SELECT p.product_name, i.quantity_on_hand, i.reorder_level
FROM inventory i
JOIN products p ON i.product_id = p.product_id
WHERE i.quantity_on_hand < i.reorder_level;
"""

df = pd.read_sql(query, engine)
df.head(4)

,product_name,quantity_on_hand,reorder_level
0,Iced Chai Frappe,23.22,26
1,Iced Green Tea Frappe,8.52,27
2,Nitro Latte,0.41,40
3,Iced Caramel Frappe,28.79,31


In [16]:
# Get the average loyalty points for customers
query = """
SELECT AVG(customer_loyalty_points) AS avg_loyalty_points
FROM customers;
"""

df = pd.read_sql(query, engine)
df.head(4)

,avg_loyalty_points
0,516.88


In [23]:
# top 5 products with the highest total sales across all orders
query_1 = """
SELECT 
    p.product_name,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_sales
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
GROUP BY p.product_name
ORDER BY total_sales DESC
LIMIT 5;
"""

# Execute the query
df_top_products = pd.read_sql(query_1, engine)
df_top_products.head()

,product_name,total_sales
0,Sumatra Coffee,5292.19
1,Eggs & Bacon,4806.35
2,Latte,4466.79
3,Green Smoothie,3827.78
4,Lemonade,3258.76


In [28]:
# products that are understocked (below the reorder level) in each branch.
query = """
SELECT 
    i.branch_id, 
    p.product_name,
    i.quantity_on_hand,
    i.reorder_level
FROM inventory i
JOIN products p ON i.product_id = p.product_id
WHERE i.quantity_on_hand < i.reorder_level
ORDER BY i.branch_id;
"""

# Execute the query
df_understocked_products = pd.read_sql(query, engine)
df_understocked_products.head()

,branch_id,product_name,quantity_on_hand,reorder_level
0,BR134603,Nitro Latte,0.41,40
1,BR254438,Cinnamon Roll,5.08,43
2,BR397515,Iced Chai Frappe,23.22,26
3,BR516213,Mocha,46.76,50
4,BR777083,Iced Green Tea Frappe,8.52,27


In [30]:
# total revenue from orders for each branch over the past 30 days
query = """
SELECT 
    o.branch_id,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_date >= CURDATE() - INTERVAL 30 DAY
GROUP BY o.branch_id;
"""

# Execute the query
df_total_revenue = pd.read_sql(query, engine)
df_total_revenue.head()

,branch_id,total_revenue
0,BR910613,2057.12
1,BR690741,92.12
2,BR974064,69.09
3,BR128808,604.94
4,BR291104,323.50


In [35]:
# suppliers with the highest-rated products
query = """
SELECT 
    s.supplier_name, 
    AVG(s.supplier_rating) AS avg_supplier_rating
FROM product_suppliers ps
JOIN suppliers s ON ps.supplier_id = s.supplier_id
JOIN products p ON ps.product_id = p.product_id
WHERE ps.is_active = 'Yes'
GROUP BY s.supplier_id
ORDER BY avg_supplier_rating DESC;
"""

# Execute the query
df_supplier_ratings = pd.read_sql(query, engine)
df_supplier_ratings.head()

,supplier_name,avg_supplier_rating
0,Global Imports,3.18
1,Dairy Masters,2.86
2,Farmers Best,1.76
3,Grain Suppliers Inc,1.41
4,Fresh Produce Ltd,1.30


In [40]:
# Average Order Value by Customer Segments
query = """
SELECT 
    CASE
        WHEN c.customer_loyalty_points < 100 THEN 'Low'
        WHEN c.customer_loyalty_points BETWEEN 100 AND 500 THEN 'Medium'
        ELSE 'High'
    END AS loyalty_segment,
    AVG(oi.quantity * oi.unit_price_at_order) AS avg_order_value
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY loyalty_segment
ORDER BY loyalty_segment;
"""

# Execute the query
df_avg_order_value_segments = pd.read_sql(query, engine)
df_avg_order_value_segments.head()

,loyalty_segment,avg_order_value
0,High,668.218286
1,Low,639.350833
2,Medium,670.003373


In [43]:
# Monthly Sales Trends for the Last Year
query = """
SELECT 
    DATE_FORMAT(o.order_date, '%Y-%m') AS month,
    SUM(oi.quantity * oi.unit_price_at_order) AS monthly_sales
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_date >= CURDATE() - INTERVAL 1 YEAR
GROUP BY month
ORDER BY month DESC;
"""

# Execute the query
df_monthly_sales_trends = pd.read_sql(query, engine)
df_monthly_sales_trends.head()

,month,monthly_sales
0,2025-04,20554.49
1,2025-03,39903.58
2,2025-02,34585.08
3,2025-01,38402.26


In [45]:
# Suppliers with the Most Active Products
query = """
SELECT 
    s.supplier_name,
    COUNT(DISTINCT p.product_id) AS active_product_count
FROM suppliers s
JOIN product_suppliers ps ON s.supplier_id = ps.supplier_id
JOIN products p ON ps.product_id = p.product_id
JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY s.supplier_id
ORDER BY active_product_count DESC;
"""

# Execute the query
df_suppliers_most_active_products = pd.read_sql(query, engine)
df_suppliers_most_active_products.head()

,supplier_name,active_product_count
0,Global Imports,44
1,Local Delights,38
2,Fresh Produce Ltd,37
3,Farmers Best,27
4,Dairy Masters,27


In [48]:
# Customer Lifetime Value (CLV) Estimation
query = """
WITH CustomerSpending AS (
    SELECT 
        c.customer_id,
        SUM(oi.quantity * oi.unit_price_at_order) AS total_spent,
        COUNT(DISTINCT o.order_id) AS order_count,
        AVG(DATEDIFF(o.order_date, o.created_at) / 30) AS avg_order_frequency
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    GROUP BY c.customer_id
)
SELECT 
    customer_id,
    total_spent,
    order_count,
    avg_order_frequency,
    total_spent / order_count * avg_order_frequency AS estimated_lifetime_value
FROM CustomerSpending
ORDER BY estimated_lifetime_value DESC;
"""

# Execute the query
df_customer_lifetime_value = pd.read_sql(query, engine)
df_customer_lifetime_value.head()

,customer_id,total_spent,order_count,avg_order_frequency,estimated_lifetime_value
0,38,69.09,1,-0.0667,-4.608303
1,139,15.76,1,-0.4333,-6.828808
2,142,14.58,1,-0.7333,-10.691514
3,110,7.64,1,-1.6333,-12.478412
4,183,112.23,1,-0.1667,-18.708741


In [51]:
#Top 5 Customers by Lifetime Spending
query = """
SELECT 
    c.customer_name,
    SUM(oi.quantity * oi.unit_price_at_order) AS lifetime_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id
ORDER BY lifetime_spent DESC
LIMIT 5;
"""

# Execute the query
df_top_customers_lifetime_spending = pd.read_sql(query, engine)
df_top_customers_lifetime_spending.head()


,customer_name,lifetime_spent
0,Darrell Solomon,2232.50
1,Daniel Prince,2162.37
2,Daniel Massey,2142.00
3,Jennifer Wells,2057.12
4,Monique Robbins,2032.00


In [ ]:
# Monthly Sales by Product and Supplier
query = """
SELECT 
    s.supplier_name,
    p.product_name,
    DATE_FORMAT(o.order_date, '%Y-%m') AS month,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_sales
FROM suppliers s
JOIN product_suppliers ps ON s.supplier_id = ps.supplier_id
JOIN products p ON ps.product_id = p.product_id
JOIN order_items oi ON p.product_id = oi.product_id
JOIN orders o ON oi.order_id = o.order_id
GROUP BY s.supplier_id, p.product_id, month
ORDER BY month DESC, total_sales DESC;
"""

# Execute the query
df_monthly_sales_by_product_supplier = pd.read_sql(query, engine)
df_monthly_sales_by_product_supplier.head()

,supplier_name,product_name,month,total_sales
0,Fresh Produce Ltd,Almond Milk Latte,2025-04,2142.00
1,Local Delights,Green Smoothie,2025-04,1864.84
2,Fresh Produce Ltd,Cinnamon Roll,2025-04,1599.17
3,Global Imports,Caramel Brulee Latte (Seasonal),2025-04,1560.15
4,Grain Suppliers Inc,Matcha Latte,2025-04,1283.50


In [ ]:
# Top Suppliers by Sales Volume
query = """
SELECT 
    s.supplier_name,
    SUM(oi.quantity) AS total_quantity_sold
FROM suppliers s
JOIN product_suppliers ps ON s.supplier_id = ps.supplier_id
JOIN products p ON ps.product_id = p.product_id
JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY s.supplier_id
ORDER BY total_quantity_sold DESC;
"""

# Execute the query
df_top_suppliers_by_sales = pd.read_sql(query, engine)
df_top_suppliers_by_sales.head()

,supplier_name,total_quantity_sold
0,Global Imports,1281.0
1,Local Delights,1006.0
2,Fresh Produce Ltd,974.0
3,Dairy Masters,734.0
4,Grain Suppliers Inc,669.0


In [ ]:
# Products with the Most Orders Across Branches
query = """
SELECT 
    p.product_name,
    SUM(oi.quantity) AS total_orders
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
GROUP BY p.product_id
ORDER BY total_orders DESC
LIMIT 10;
"""

# Execute the query
df_most_ordered_products = pd.read_sql(query, engine)
df_most_ordered_products.head()

,product_name,total_orders
0,Mocha,50.0
1,Croissant (Plain),50.0
2,Caramel Milkshake,50.0
3,Almond Milk Latte,50.0
4,Muffin (Blueberry),50.0


In [ ]:
# Customer Orders Grouped by Order Status and Branch
query = """
SELECT 
    b.branch_name,
    o.order_status,
    COUNT(o.order_id) AS total_orders
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN branches b ON o.branch_id = b.branch_id
GROUP BY b.branch_id, o.order_status
ORDER BY total_orders DESC;
"""

# Execute the query
df_orders_by_status_branch = pd.read_sql(query, engine)
df_orders_by_status_branch.head()

,branch_name,order_status,total_orders
0,ABC Place,Cancelled,1
1,Java House Lavington,In Progress,1
2,Java House Lavington,Pending,1
3,Java House Lenana Road,Pending,1
4,Java House Mama Ngina Street,Pending,1


In [ ]:
# Supplier Performance by Product and Sales Value
query = """
SELECT 
    s.supplier_name,
    p.product_name,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_sales
FROM suppliers s
JOIN product_suppliers ps ON s.supplier_id = ps.supplier_id
JOIN products p ON ps.product_id = p.product_id
JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY s.supplier_id, p.product_id
ORDER BY total_sales DESC;
"""

# Execute the query
df_supplier_performance_sales = pd.read_sql(query, engine)
df_supplier_performance_sales.head()

,supplier_name,product_name,total_sales
0,Local Delights,Eggs & Bacon,2232.50
1,Local Delights,Sumatra Coffee,2162.37
2,Fresh Produce Ltd,Almond Milk Latte,2142.00
3,Dairy Masters,Bagel (Cinnamon Raisin),2057.12
4,Global Imports,Mocha,2032.00


In [ ]:
# Customer Purchase Patterns Over Time
query = """
SELECT 
    c.customer_name,
    DATE_FORMAT(o.order_date, '%Y-%m') AS month,
    COUNT(o.order_id) AS total_orders,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id, month
ORDER BY c.customer_name, month DESC;
"""

# Execute the query
df_customer_purchase_patterns = pd.read_sql(query, engine)
df_customer_purchase_patterns.head()


,customer_name,month,total_orders,total_sales
0,Albert Elliott MD,2025-03,1,1135.86
1,Alejandra Powell,2025-01,1,1524.80
2,Alexander Morales,2025-03,1,452.14
3,Alexandra Martinez,2025-04,1,604.94
4,Alexis Goodwin,2025-03,1,374.50


In [ ]:
# Customer Purchase History with Product Categories
query = """
SELECT 
    c.customer_name,
    c.customer_email,
    p.category_name,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY c.customer_id, p.category_name
ORDER BY total_spent DESC;
"""
df_customer_purchase_history = pd.read_sql(query, engine)
df_customer_purchase_history.head()

,customer_name,customer_email,category_name,total_orders,total_spent
0,Darrell Solomon,april@yahoo.com,Snacks,1,2232.50
1,Daniel Prince,peter@test.io,Snacks,1,2162.37
2,Daniel Massey,brent@gmail.com,Flavored Coffees,1,2142.00
3,Jennifer Wells,robert@company.com,Specialty Drinks,1,2057.12
4,Monique Robbins,william@company.com,Teas,1,2032.00


In [ ]:
# top 5 best-selling products per branch
query= """
SELECT 
    branch_name,
    product_name,
    total_sales
FROM (
    SELECT 
        b.branch_name,
        p.product_name,
        SUM(oi.quantity * oi.unit_price_at_order) AS total_sales,
        (
            SELECT COUNT(*)
            FROM (
                SELECT 
                    SUM(oi2.quantity * oi2.unit_price_at_order) AS branch_product_sales
                FROM order_items oi2
                JOIN orders o2 ON oi2.order_id = o2.order_id
                WHERE o2.branch_id = o.branch_id
                AND oi2.product_id != oi.product_id
                GROUP BY oi2.product_id
                HAVING branch_product_sales > SUM(oi.quantity * oi.unit_price_at_order)
            ) AS higher_sales
        ) AS sales_rank
    FROM order_items oi
    JOIN products p ON oi.product_id = p.product_id
    JOIN orders o ON oi.order_id = o.order_id
    JOIN branches b ON o.branch_id = b.branch_id
    GROUP BY b.branch_name, p.product_name, o.branch_id, oi.product_id
) ranked_products
WHERE sales_rank < 5 OR sales_rank IS NULL
ORDER BY branch_name, sales_rank;
"""
df_top_n_products_by_branch = pd.read_sql(query, engine)
df_top_n_products_by_branch.head()

,branch_name,product_name,total_sales
0,ABC Place,Toffee Nut Mocha (Seasonal),127.65
1,ABC Place,Wrap (Veggie),86.66
2,ABC Place,Cold Brew,84.15
3,ABC Place,Gingerbread Latte (Seasonal),474.47
4,ABC Place,French Toast,705.90


In [ ]:
# top-performing branches per product
query = """
SELECT 
    product_name,
    branch_name,
    total_sales
FROM (
    SELECT 
        p.product_name,
        b.branch_name,
        SUM(oi.quantity * oi.unit_price_at_order) AS total_sales,
        (
            SELECT COUNT(*)
            FROM (
                SELECT 
                    SUM(oi2.quantity * oi2.unit_price_at_order) AS product_branch_sales
                FROM order_items oi2
                JOIN orders o2 ON oi2.order_id = o2.order_id
                WHERE oi2.product_id = oi.product_id
                AND o2.branch_id != o.branch_id
                GROUP BY o2.branch_id
                HAVING product_branch_sales > SUM(oi.quantity * oi.unit_price_at_order)
            ) AS higher_sales
        ) AS sales_rank
    FROM order_items oi
    JOIN products p ON oi.product_id = p.product_id
    JOIN orders o ON oi.order_id = o.order_id
    JOIN branches b ON o.branch_id = b.branch_id
    GROUP BY p.product_name, b.branch_name, oi.product_id, o.branch_id
) ranked_branches
WHERE sales_rank < 5 OR sales_rank IS NULL
ORDER BY product_name, sales_rank;
"""
df_top_n_branches_by_product = pd.read_sql(query, engine)
df_top_n_branches_by_product.head()

,product_name,branch_name,total_sales
0,Affogato,Java House Parklands Road,1228.53
1,Affogato,Java House Landside,80.80
2,Affogato,Java House Lunga Lunga,15.76
3,Affogato,Java House Ciata Mall,382.21
4,Affogato,Java House Kericho,899.64


In [ ]:
# Total orders and total spend per customer
query = """
SELECT 
    c.customer_name,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id
ORDER BY total_spent DESC;
"""
df_total_spend = pd.read_sql(query, engine)
df_total_spend.head()

,customer_name,total_orders,total_spent
0,Darrell Solomon,1,2232.50
1,Daniel Prince,1,2162.37
2,Daniel Massey,1,2142.00
3,Jennifer Wells,1,2057.12
4,Monique Robbins,1,2032.00


In [ ]:
# Most frequently ordered product per customer
query = """
SELECT 
    c.customer_name,
    p.product_name,
    SUM(oi.quantity) AS total_quantity
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY c.customer_id, p.product_id
ORDER BY total_quantity DESC;
"""
df_top_products = pd.read_sql(query, engine)
df_top_products.head()

,customer_name,product_name,total_quantity
0,Joseph Walker,Caramel Milkshake,50.0
1,Shelly Hays,Croissant (Plain),50.0
2,Monique Robbins,Mocha,50.0
3,Daniel Massey,Almond Milk Latte,50.0
4,Erika Hodges,Muffin (Blueberry),50.0


In [ ]:
# Total orders and spend per customer per branch
query = """
SELECT 
    c.customer_name,
    b.branch_name,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(oi.quantity * oi.unit_price_at_order) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN branches b ON o.branch_id = b.branch_id
GROUP BY c.customer_id, b.branch_id
ORDER BY total_spent DESC;
"""
df_branch_spending = pd.read_sql(query, engine)
df_branch_spending.head()

,customer_name,branch_name,total_orders,total_spent
0,Darrell Solomon,"Java House Centre Point, Diani Beach Rd",1,2232.50
1,Daniel Prince,Java House Junction,1,2162.37
2,Daniel Massey,Java House Lunga Lunga,1,2142.00
3,Jennifer Wells,Java House Lavington,1,2057.12
4,Monique Robbins,"Java House Centre Point, Diani Beach Rd",1,2032.00


In [ ]:
# First and last order date per customer
query = """
SELECT 
    c.customer_name,
    MIN(o.order_date) AS first_order,
    MAX(o.order_date) AS last_order
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id;
"""
df_order_dates = pd.read_sql(query, engine)
df_order_dates.head()

,customer_name,first_order,last_order
0,Jeffrey Gonzalez,2025-03-14,2025-03-14
1,Jennifer Wells,2025-03-26,2025-03-26
2,William Kim,2025-01-28,2025-01-28
3,Amanda Garcia,2025-04-11,2025-04-11
4,Briana Simmons,2025-02-26,2025-02-26


In [ ]:
# Monthly order count per customer
query = """
SELECT 
    c.customer_name,
    DATE_FORMAT(o.order_date, '%Y-%m') AS order_month,
    COUNT(o.order_id) AS monthly_orders
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, order_month
ORDER BY c.customer_name, order_month;
"""
df_monthly_orders = pd.read_sql(query, engine)
df_monthly_orders.head()

,customer_name,order_month,monthly_orders
0,Albert Elliott MD,2025-03,1
1,Alejandra Powell,2025-01,1
2,Alexander Morales,2025-03,1
3,Alexandra Martinez,2025-04,1
4,Alexis Goodwin,2025-03,1
